In [1]:
import pandas as pd                                                               #(path:- /Users/haresh.sam/Documents/tempV/table.csv)
from sentence_transformers import SentenceTransformer
from fuzzywuzzy import fuzz
import numpy as np
import time

start_time = time.time()

def load_data(file_path):
    return pd.read_csv(file_path).head(100)

def lowercase_all(df):
    return df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

def generate_embeddings(model, df):
    embeddings = {}
    for column in df.columns:
        column_embeddings = model.encode(df[column].astype(str).tolist(), convert_to_tensor=True)
        embeddings[column] = column_embeddings.cpu().numpy()
    return embeddings

def compute_cosine_similarity(query_embedding, entry_embeddings):
    similarities = np.dot(entry_embeddings, query_embedding.T)
    return similarities.max()

def calculate_similarities(df, query, model):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()
    results = []
    embeddings = generate_embeddings(model, df)
    
    for index, row in df.iterrows():
        row_similarities = []
        for column in df.columns:
            entry_embeddings = embeddings[column][index].reshape(1, -1)
            semantic_similarity = compute_cosine_similarity(query_embedding, entry_embeddings)
            
            # Fuzzy logic match
            fuzzy_similarity = fuzz.ratio(query, str(row[column]))
            fuzzy_similarity /= 100  # Normalize fuzzy score to [0, 1] range
            
            # Combine both similarities
            combined_similarity = (semantic_similarity + fuzzy_similarity) / 2
            row_similarities.append((column, combined_similarity))
        
        max_similarity = max(row_similarities, key=lambda x: x[1])[1]
        results.append((index, max_similarity))
    
    return results

def find_best_match(similarities):
    best_match = max(similarities, key=lambda x: x[1], default=None)
    return best_match

def process_data(file_path, query):
    df = load_data(file_path)
    df = lowercase_all(df)
    model = SentenceTransformer('all-MiniLM-L6-v2')
    similarities = calculate_similarities(df, query, model)
    best_match = find_best_match(similarities)
    return best_match, df

def main():
    file_path = input("Enter the path to the CSV file: ")
    user_query = input("Enter the query to match: ")
    best_match, df = process_data(file_path, user_query)
    
    if best_match is not None:
        index, similarity = best_match
        print("File path:", file_path)
        print("Query given:", user_query)
        print("Best Match Found at index:", index)
        print("Similarity Score:", similarity)
        print(df.iloc[index])
    else:
        print("No match found")

if __name__ == "__main__":
    main()

end_time = time.time()
runtime = end_time - start_time
print(f"Runtime: {runtime} seconds")


/Users/haresh.sam/Documents/tempV/venv_flower/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/haresh.sam/Documents/tempV/venv_flower/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/haresh.sam/Documents/tempV/venv_flower/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/var/folders/54/v3l5y7rn66b76fdzslzssp3w0000gn/T/ipykernel_43783/2613921748.py:13

File path: /Users/haresh.sam/Documents/tempV/table.csv
Query given: Black man
Best Match Found at index: 6
Similarity Score: 0.5470053815841674
title                                 black man in the netherlands
author                                          francio guadeloupe
category                                              anthropology
publish_year                               first published in 2022
title_id                                        /works/ol25915217w
author_id                   /authors/ol8236241a/francio_guadeloupe
cover_url        //covers.openlibrary.org/b/olid/ol34977443m-m.jpg
book_stats       publish date 2022|publisher  university press ...
descriptions     this edition doesn't have a description yet. c...
reading_stats       0 want to read|0 currently reading|0 have read
Name: 6, dtype: object
Runtime: 40.82295489311218 seconds
